change from V0:<br>
look at three points before deciding what to drop:
1. colinearity
2. features unable to normalize
3. P-value and R-squared values

# Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None) # shows me all columns instead of truncating and dot dot dot

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

from scipy.stats import kurtosis, skew
import scipy.stats as stats

from math import sqrt

import statsmodels.formula.api as smf

##### pull in data and inspect dataframe

In [3]:
df = pd.read_csv("cleaned_kc_housing_data.csv")
df.head()

,Unnamed: 0,date_sold,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,10/13/2014,221900,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,12/9/2014,538000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,2/25/2015,180000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,3,12/9/2014,604000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,4,2/18/2015,510000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21534 entries, 0 to 21533
Data columns (total 21 columns):
Unnamed: 0       21534 non-null int64
date_sold        21534 non-null object
price            21534 non-null int64
bedrooms         21534 non-null int64
bathrooms        21534 non-null float64
sqft_living      21534 non-null int64
sqft_lot         21534 non-null int64
floors           21534 non-null float64
waterfront       21534 non-null int64
view             21534 non-null int64
condition        21534 non-null int64
grade            21534 non-null int64
sqft_above       21534 non-null int64
sqft_basement    21534 non-null int64
yr_built         21534 non-null int64
yr_renovated     21534 non-null int64
zipcode          21534 non-null int64
lat              21534 non-null float64
long             21534 non-null float64
sqft_living15    21534 non-null int64
sqft_lot15       21534 non-null int64
dtypes: float64(4), int64(16), object(1)
memory usage: 3.5+ MB


#### label encode categorical data

In [6]:
#heres my categorical data
categorical = df[['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade']]

In [8]:


from sklearn.preprocessing import LabelEncoder  # import LabelEncoder
lb_make = LabelEncoder()  # creating variable to perform action -- why? brevity or serves a greater purpose?

In [12]:
encoded = pd.DataFrame()
encoded["bedrooms"] = lb_make.fit_transform(categorical.bedrooms)
encoded["bathrooms"] = lb_make.fit_transform(categorical.bathrooms)
encoded["floors"] = lb_make.fit_transform(categorical.floors)
encoded["waterfront"] = lb_make.fit_transform(categorical.waterfront)
encoded["view"] = lb_make.fit_transform(categorical.view)
encoded["condition"] = lb_make.fit_transform(categorical.condition)
encoded["grade"] = lb_make.fit_transform(categorical.grade)

In [13]:
encoded

,bedrooms,bathrooms,floors,waterfront,view,condition,grade
0,2,2,0,0,0,2,4
1,2,7,2,0,0,2,4
2,1,2,0,0,0,2,3
3,3,10,0,0,0,4,4
4,2,6,0,0,0,2,5
5,3,16,0,0,0,2,8
6,2,7,2,0,0,2,4
7,2,2,0,0,0,2,4
8,2,8,2,0,0,2,4
9,2,8,0,0,0,2,5


#### run OLS on label encoded categorical data

In [ ]:
cols_store= walmart.columns[7:16]
cols_dept = walmart.columns[17:95]
cols_type = walmart.columns[96:97]
cols_holiday = walmart.columns[1:2] 

cols = [cols_store, cols_dept , cols_type, cols_holiday]
for col in cols:
    sum_cols = "+".join(col)
    form = "Weekly_Sales ~" + sum_cols
    model = smf.ols(formula= form, data= walmart).fit()
    print(model.summary())